In [5]:
%cd ..
%matplotlib inline

C:\Users\usuario\Desktop\New_Work


In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import re
import unicodedata

In [7]:
from src.common_paths import get_data_path, get_output_path
from src.common_paths import get_data_path, get_output_path
from src.utilities import *

In [8]:
symbols_to_space = re.compile(u"[/\|\n(\; )|(\: )|( \()|(\) )|( \")|(\" )|( \')|(\' )]")
symbols_to_remove = re.compile(u"[\"\'\$\€\£\(\)\:\[\]\.\,\-]")
space_repetition = re.compile(u" {2,}")
key_words_to_remove = re.compile(u"gmbh")

def canonize_language(df, text_var):
    text_var_clean= text_var+'_clean'
    cleaned_var=df[text_var].apply(lambda x: re.sub(symbols_to_remove, "", x))
    cleaned_var=cleaned_var.apply(lambda x: x.lower())
    cleaned_var=cleaned_var.apply(lambda x: re.sub(key_words_to_remove, "", x))
    cleaned_var=cleaned_var.apply(lambda x: unicodedata.normalize('NFKD', x))
    cleaned_var=cleaned_var.apply(lambda x: re.sub(space_repetition, "", x))
    cleaned_var=cleaned_var.apply(lambda x: str.strip(x))
    cleaned_var=cleaned_var.apply(lambda x: re.sub(u"\s", "_", x))
    return cleaned_var


def match_checker(df1, df2, key, aux_how):
    df2["aux"] = 1
    aux_df = pd.merge(df1, df2, on = key, how = aux_how)
    aux_df["aux"] = aux_df.aux.fillna(0)
    num_ids_match= np.sum(aux_df.aux)
    print("{} var has {} unique values".format(key, len(np.unique(df1[key]))))
    print("{} var has {} unique values".format(key, len(np.unique(df2[key]))))
    print("Matches {} id rows of entities in profiles".format(num_ids_match))
    pass



def direct_matcher(df1, df2, key, aux_how):
    aux_df = pd.merge(df1, df2, on = key, how = aux_how)
    aux_df=aux_df[["id_x", "id_y"]]
    aux_df.rename(columns={"id_y": "id_profiles", "id_x": "id_entities"}, inplace=True)
    return(aux_df[["id_profiles", "id_entities"]])


def eval_fun(profile_ids, aux_vars_ids, matched_df):
    # The evaluation will count if all of the profiles have at least 1 entity
    # Global and local mean of % matching entitites with profile.
    gt_df = pd.read_csv(os.path.join(get_data_path(), "ground_truth.tsv"), 
                                names=aux_vars_ids,
                                encoding="utf-8", 
                                sep="\t") # .fillna({"text": "empty"})


    # First check: ¿At least all the profiles have one entity?
    at_least_all=len(np.unique(gt_df[aux_vars_ids[0]])) == len(np.unique(matched_df[aux_vars_ids[0]]))
    if at_least_all:
        print("1. All profiles have at least one entity")
    else:
        print("1. Not all profiles have one entity")
    # Second is to measure the percentages:
    res_df = pd.DataFrame() # DF with a tuple [a, b, c] where a= # of entitites assigned, b= # of real assigned, c= #matches 
    for i_id in profile_ids:
        entities_assigned=matched_df.loc[matched_df["id_profiles"]==i_id].id_entities.values
        entities_assigned = entities_assigned[~np.isnan(entities_assigned)]
        num_assigned= len(entities_assigned)
        real_entities_assigned=gt_df.loc[gt_df["id_profiles"]==i_id].id_entities.values
        real_entities_assigned = real_entities_assigned[~np.isnan(real_entities_assigned)]
        real_num_assigned=len(real_entities_assigned)
        num_matches=len(set(entities_assigned) & set(real_entities_assigned))
        per_match= round(num_matches/real_num_assigned, 2)*100
        aux_tuple=[i_id, num_assigned, real_num_assigned, per_match]
        aux_df=pd.DataFrame([aux_tuple], columns=["id_profile", "num_entities_matched", "real_entitites_matched", "per_match"])
        res_df=res_df.append(aux_df) 
    return(res_df)

In [9]:
entities_df = pd.read_csv(os.path.join(get_output_path(), "enti_data.csv"), 
                            encoding="utf-8", 
                            sep=";") # .fillna({"text": "empty"})

profiles_df = pd.read_csv(os.path.join(get_output_path(), "prof_data.csv"), 
                            encoding="utf-8", 
                            sep=";") # .fillna({"text": "empty"})

In [10]:
# Standarized ID company_name 
entities_df['company_name_clean']=canonize_language(df=entities_df, text_var="company_name")
profiles_df['company_name_clean']=canonize_language(df=profiles_df, text_var="company_name")

In [14]:
print(profiles_df.columns)
print(entities_df.columns)

Index(['id', 'company_name', 'city', 'country', 'foundation_year_cat',
       'company_name_clean'],
      dtype='object')
Index(['id', 'company_name', 'city', 'country', 'foundation_year_cat',
       'company_name_clean'],
      dtype='object')


In [15]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

def fuzzy_merge(df_1, df_2, key1, key2, threshold=90, limit=2):
    """
    :param df_1: the left table to join
    :param df_2: the right table to join
    :param key1: key column of the left table
    :param key2: key column of the right table
    :param threshold: how close the matches should be to return a match, based on Levenshtein distance
    :param limit: the amount of matches that will get returned, these are sorted high to low
    :return: dataframe with boths keys and matches
    """
    s = df_2[key2].tolist()

    m = df_1[key1].apply(lambda x: process.extract(x, s, limit=limit))    
    df_1['matches'] = m

    m2 = df_1['matches'].apply(lambda x: ', '.join([i[0] for i in x if i[1] >= threshold]))
    df_1['matches'] = m2

    return df_1

In [ ]:
fuzzy_merge(profiles_df, entities_df, 'company_name', 'company_name', threshold=80) # TOO MUCH TIME CONSUMING